In [2]:
# Add parent folder to path
import sys
sys.path.append("..")

# External libraries
from tqdm.autonotebook import tqdm
import scipy.spatial
import numpy as np

# Own
from src.Kernels.Gaussian import Gaussian
from src.Particle import Particle
from src.Equations.WCSPH import WCSPH
from src.Animation import Animation
from src.Integrators.EulerIntegrater import EulerIntegrater

# Data loading
from pysph.solver.utils import load

C:\Users\Jasper\AppData\Local\Programs\Python\Python37-32\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
def create_particles():
    # Load PySPH particles
    data = load('dam_break_2d_0.npz')

    # Convert boundaries to particles
    particles = []
    for t in ['boundary', 'fluid']:
        for i in range(len(data['arrays'][t].x)):
            x = data['arrays'][t].x[i]
            y = data['arrays'][t].y[i]
            rho = data['arrays'][t].rho[i]
            particles.append(Particle(x=x, y=y, rho=rho, label=t))    

    # Set constant mass
    mass = data['arrays'][t].m[0]
    return particles, mass

In [3]:
def calc_smoothing_length(mass, rho_a: np.array) -> np.array:
    return 1.0 * np.sqrt(mass / rho_a)

In [4]:
# Runs a (2D) dambreak example
# WCSPH method
wcsph = WCSPH(height=2.0)

# Generate some particles
# N * N (water-particles).
print('Creating Particles.')
particles, mass = create_particles()
fluid_particles = [p for p in particles if p.label == 'fluid']
print(f'Created {len(fluid_particles)} fluid-particles.')
print(f'Created {len(particles) - len(fluid_particles)} boundary-particles.')
print(f'For a total of {len(particles)} particles.')

# Get fluid indices
fluid_ind = []
for i in range(len(particles)):
    if particles[i].label == 'fluid':
        fluid_ind.append(i)
fluid_ind = np.array(fluid_ind)

# Solving properties
kernel = Gaussian()
integrater = EulerIntegrater()

# Time-step properties
t_max = 0.5 # [s]
dt = 0.01   # [s]
t = np.arange(0, t_max, dt)
t_n = len(t)

# water column height
H = wcsph.H # [m]

# H-parameter
#hdx = 1.3 * np.sqrt(H * 2 * 2 / len(fluid_particles))
hdx = 0.039
h = np.ones(len(particles)) * hdx

# Gravity
gx = 0.
gy = -9.81

# Initialize the loop
for p in particles:
    wcsph.inital_condition(p)

# Time-stepping
x = np.zeros([len(particles), t_n])  # X-pos
y = np.zeros([len(particles), t_n])  # Y-pos
c = np.zeros([len(particles), t_n])  # Pressure (p)
u = np.zeros([len(particles), t_n])  # Density (rho)
v = np.zeros([len(particles), 2, t_n])  # Velocity both x and y
q = np.zeros([len(particles), 2, t_n]) # Acceleration both x and y

# Initialization
x[:, 0] = [p.r[0] for p in particles]
y[:, 0] = [p.r[1] for p in particles]
c[:, 0] = [p.p for p in particles]
u[:, 0] = [p.rho for p in particles]

# Integration loop
# TODO: Move to separate (generic) function
i: int = 0
for t_step in tqdm(range(t_n - 1), desc='Time-stepping'):
    # Distance and neighbourhood
    r = np.array([p.r for p in particles])
    dist = scipy.spatial.distance.cdist(r, r, 'euclidean')
    hood = scipy.spatial.cKDTree(r)

    # Force/Acceleration evaluation loop
    i: int = 0
    for p in tqdm(particles, desc='Evaluating equations', leave=False):
        # Set acceleration etc to zero.
        wcsph.loop_initialize(p)

        # Query neighbours
        r_dist: float = np.min(3.01 * h)  # Goes to zero when q > 3
        near_ind: list = hood.query_ball_point(p.r, r_dist)
        near_arr: np.array = np.array(near_ind)

        # Calculate some general properties
        xij: np.array = r[near_arr] - p.r
        rij: np.array = dist[i, near_arr]
        vij: np.array = v[near_arr, :, t_step] - p.v
        dwij: np.array = kernel.gradient(xij, rij, h[near_arr])

        # Evaluate the equations
        wcsph.TaitEOS(p)
        wcsph.Continuity(mass, p, xij, rij, dwij, vij)
        
        if p.label == 'fluid':
            wcsph.Momentum(mass, p, c[near_arr, t_step],
                           u[near_arr, t_step], dwij)
            wcsph.Gravity(p, gx, gy)
        i += 1

    # Integration loop
    i: int = 0
    for p in tqdm(particles, desc='Integration', leave=False):
        # Integrate the thingies
        integrater.integrate(dt, p)

        # Put into giant-matrix
        x[i, t_step + 1] = p.r[0]
        y[i, t_step + 1] = p.r[1]
        c[i, t_step + 1] = p.p
        u[i, t_step + 1] = p.rho
        v[i, :, t_step + 1] = p.v
        q[i, :, t_step + 1] = p.a

        i += 1
        
    # Calculate the new smoothing length
    h[fluid_ind] = calc_smoothing_length(mass, u[fluid_ind, t_step + 1])
    
print('Integration complete.')
# END Integration loop

# Export
print('Starting export.')
fluid_ind = []
solid_ind = []
i: int = 0
for p in particles:
    if p.label == 'fluid':
        fluid_ind.append(i)
    else:
        solid_ind.append(i)
    i += 1
Animation(x=x[fluid_ind], y=y[fluid_ind], r=3.0, c=c[fluid_ind], fps=20, xlim=[-1, 6], ylim=[-1, 6], xsolid=x[solid_ind], ysolid=y[solid_ind]).export(f'pysph.mp4')
print('Export complete.')

Creating Particles.
Created 4489 fluid-particles.
Created 803 boundary-particles.
For a total of 5292 particles.


C:\Users\Jasper\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  


C:\Users\Jasper\AppData\Local\Programs\Python\Python37-32\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


IndexError: arrays used as indices must be of integer (or boolean) type

In [18]:
h[5289]

nan

In [20]:
u[5289, :]

array([1000.16242084, 1000.16242084,  984.48076514,  927.66556999,
       -237.94082336,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ])

In [16]:
5291 - 3

5288

In [21]:
r_dist

nan

In [32]:
data = load('dam_break_2d_2000.npz')
data['arrays']['fluid'].arho

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
dat1 = load('dam_break_2d_0.npz')
dat2 = load('dam_break_2d_100.npz')
delta = dat1['arrays']['fluid'].rho - dat2['arrays']['fluid'].rho
delta

array([-5.98715304e-01, -8.11459070e-01, -8.53415890e-01, ...,
       -1.20053301e-10, -8.27640179e-11, -4.46789272e-11])

In [18]:
delta / dat2['solver_data']['t']

array([-6.01352030e+01, -8.15032713e+01, -8.57174309e+01, ...,
       -1.20582012e-08, -8.31285082e-09, -4.48756920e-09])

In [21]:
dat2['solver_data']['t']

0.009956153383402078